In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import pandas as pd
from datetime import datetime
import altair as alt
%config Completer.use_jedi = False

In [3]:
BASE_PATH = '../../covid19-opendata-vaccini/dati'
AGE_ORDER = ['12-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']
FOOTNOTE = 'Grafico @RaffoRaffo85 | Dati aggiornati {}'.format(datetime.today().strftime('%Y-%m-%d'))

In [4]:
def get_tot_per_region(df):
    tt_df = (
        df
        .groupby('area', as_index=False)
        .totale_popolazione
        .sum()
        .rename(columns={'totale_popolazione': 'tot_reg'})
    )
    return df.merge(tt_df, on='area', how='left')

In [5]:
demography_region_df = (
    pd.read_csv(os.path.join(BASE_PATH, 'platea.csv'))
    .pipe(get_tot_per_region)
    .assign(share=lambda x: x.totale_popolazione/x.tot_reg)
)
demography_all_df = (
    demography_region_df
    .groupby('fascia_anagrafica', as_index=False)
    .totale_popolazione
    .sum()
    .assign(share=lambda x: x.totale_popolazione/x.totale_popolazione.sum())
)

In [6]:
(
    alt.Chart(demography_all_df)
    .mark_bar()
    .encode(
        x=alt.X('share:Q', axis=alt.Axis(format='.0%'), title='Percentuale popolazione'),
        y=alt.Y('fascia_anagrafica:N', title='Fascia d\'etá')
    )
)

alt.Chart(...)

In [7]:
(
    alt.Chart(demography_region_df, height=20, width=480)
    .mark_circle(size=30)
    .encode(
        y=alt.Y('jitter:Q', title=None, axis=alt.Axis(ticks=True, grid=False, labels=False)),
        x=alt.X('share:Q', axis=alt.Axis(format='.0%'), title='Percentuale popolazione, per regione'),
        #color=alt.Color('fascia_anagrafica:N', legend=None),
        row=alt.Row(
            'fascia_anagrafica:N', title='',
            header=alt.Header(labelAngle=0, labelPadding=-16, labelFontWeight='bold'),
        ),
        tooltip=[alt.Tooltip('nome_area:N'), alt.Tooltip('fascia_anagrafica:N'), alt.Tooltip('share:Q', format='.0%')]
    )
    .transform_calculate(jitter='sqrt(-2*log(random()))*cos(2*PI*random())')
    .configure_facet(spacing=3)
    .configure_view(stroke=None)
)

alt.Chart(...)

In [8]:
(
    alt.Chart(demography_region_df)
    .mark_bar()
    .encode(
        x=alt.X('share:Q', axis=alt.Axis(format='.0%'), scale=alt.Scale(domain=[0, 1]), title='Percentuale popolazione'),
        y=alt.Y('nome_area:N', title='Regione'),
        color=alt.Color('fascia_anagrafica:N', title='Fascia d\'etá')
    )
)

alt.Chart(...)